# Fantasy Football Analysis
Utilizing the yahoo-fantasy-api, this notebook explores data from the fantasy football league to draw analytical conclusions.

Start by pulling in the necessary libraries and generating OAuth2 configuration from Yahoo's API.

In [ ]:
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
# import yff
# import google_sheets as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
import difflib
%matplotlib inline

from importlib import reload
# reload(gs)
# reload(yff)

pd.set_option('max_row', None)

oauth = OAuth2(None, None, from_file='yahoo.json')

## Explore 2020 Season

In [ ]:
# reload(yff)

# Create season object
# ssn20 = yff.fantasy_season(oauth, 2020)

In [ ]:
# ssn20.get_team_data('9')

## Draft Prep
Pull Player Data and Rankings From Various Websites

In [40]:
replacements = {'Pat Mahomes': 'Patrick Mahomes', 
                'Allen Robinson II': 'Allen Robinson',
                'D.K. Metcalf': 'DK Metcalf',
                'J.K. Dobbins': 'JK Dobbins',
                'D.J. Moore': 'DJ Moore',
                'A.J. Brown': 'AJ Brown',
                'A.J. Dillon': 'AJ Dillon',
                'Ben Roethlisberger PIT': 'Ben Roethlisberger',
                'D.J. Chark': 'DJ Chark',
                'DJ Chark Jr.': 'DJ Chark',
                'Darrell Henderson Jr.': 'Darrell Henderson',
                'J.D. McKissic': 'JD McKissic',
                'Justin Fields CHI': 'Justin Fields',
                'Kirk Cousins MIN': 'Kirk Cousins',
                'Marvin Jones Jr.': 'Marvin Jones',
                'Melvin Gordon III': 'Melvin Gordon',
                'Michael Pittman Jr.': 'Michael Pittman',
                'Odell Beckham Jr': 'Odell Beckham',
                'Odell Beckham Jr.': 'Odell Beckham',
                'Robert Tonyan Jr.': 'Robert Tonyan',
                'Ronald Jones II': 'Ronald Jones',
                'Teddy Birdgewater DEN': 'Teddy Birdgewater',
                'Trey Lance SF': 'Trey Lance',
                'Will Fuller V': 'Will Fuller',
                'Zack Wilson NYJ': 'Zach Wilson',
                }

espn_teams =   {'Carolina Panthers': 'CAR',
                'Kansas City Chiefs': 'KC',
                'Minnesota Vikings': 'MIN',
                'New Orleans Saints': 'NO',
                'Buffalo Bills': 'BUF',
                'Tennessee Titans': 'TEN',
                'Baltimore Ravens': 'BAL',
                'Dallas Cowboys': 'DAL',
                'Arizona Cardinals': 'ARI',
                'Green Bay Packers': 'GB',
                'New York Giants': 'NYG',
                'Seattle Seahawks': 'SEA',
                'Los Angeles Chargers': 'LAC',
                'Indianapolis Colts': 'IND',
                'Cleveland Browns': 'CLE',
                'Atlanta Falcons': 'ATL',
                'Washington Football Team': 'WAS',
                'Tampa Bay Buccaneers': 'TB',
                'Pittsburgh Steelers': 'PIT',
                'Cincinnati Bengals': 'CIN',
                'Las Vegas Raiders': 'LV',
                'Chicago Bears': 'CHI',
                'San Francisco 49ers': 'SF',
                'Los Angeles Rams': 'LAR',
                'Detroit Lions': 'DET',
                'Philadelphia Eagles': 'PHI',
                'Miami Dolphins': 'MIA',
                'Jacksonville Jaguars': 'JAX',
                'Denver Broncos': 'DEN',
                'New York Jets': 'NYJ',
                'New England Patriots': 'NE',
                'Houston Texans': 'HOU'}
sn_teams = {'Panthers': 'CAR',
            'Chiefs': 'KC',
            'Vikings': 'MIN',
            'Saints': 'NO',
            'Bills': 'BUF',
            'Titans': 'TEN',
            'Ravens': 'BAL',
            'Cowboys': 'DAL',
            'Cardinals': 'ARI',
            'Packers': 'GB',
            'Giants': 'NYG',
            'Seahawks': 'SEA',
            'Chargers': 'LAC',
            'Colts': 'IND',
            'Browns': 'CLE',
            'Falcons': 'ATL',
            'Washington': 'WAS',
            'Buccaneers': 'TB',
            'Steelers': 'PIT',
            'Bengals': 'CIN',
            'Raiders': 'LV',
            'Bears': 'CHI',
            '49ers': 'SF',
            'Rams': 'LAR',
            'Lions': 'DET',
            'Eagles': 'PHI',
            'Dolphins': 'MIA',
            'Jaguars': 'JAX',
            'Broncos': 'DEN',
            'Jets': 'NYJ',
            'Patriots': 'NE',
            'Texans': 'HOU'}

# Sporting News
def sn_df():
    url = 'https://www.sportingnews.com/us/fantasy/news/fantasy-superflex-rankings-2021-top-200-ppr/ze1ic3xfl0gd1eptktn9jkodm'
    df = pd.read_html(url, header=0)[0]
    split_df = df['Player'].str.split(',', expand=True)
    df['Player'] = split_df[0]
    df['Team'] = split_df[1]
    df = df.rename({'Rank': 'SN Rank'}, axis=1)
    df['Team'] = df['Team'].str.strip()

    # Create Ranks
    pos_rank = df.groupby('Position')['SN Rank'].rank().astype('int')
    df['SN Pos'] = df['Position'] + pos_rank.astype('str')
    df = df.drop('Position', axis=1)
    df = df.replace(replacements).replace(sn_teams).replace(espn_teams)
    return df

# ESPN
def espn_df():
    url = 'https://www.espn.com/fantasy/football/story/_/id/32008027/tristan-h-cockcroft-updated-2021-fantasy-football-rankings-superflex-2-qb-leagues'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    para = ["Player", "Eligible Pos", "Team", "Pos Rank"]
    ls = soup.find_all('p')[2].text
    ls = re.split(r'\n', ls)  # remove newline characters
    df = pd.DataFrame([re.split(r'(?:\A\w+\.)|(?:,)', txt) for txt in ls]).drop(0, axis=1).drop(0, axis=0)
    df.columns = para
    df['Player'] = df['Player'].str.strip()
    df['Team'] = df['Team'].str.strip()
    df = df.reset_index().rename({'Eligible Pos': 'Position', 'index': 'ESPN Rank', 'Pos Rank': 'ESPN Pos'}, axis=1)
    df = df.drop('Position', axis=1)
    df = df.replace(replacements).replace(sn_teams).replace(espn_teams)
    return df

# CBS Sports
def cbs_df():
    df = pd.read_csv(r"cbs.csv", header=None)
    df['Position'] = df[0].str.extract(r'\((\w+)\)')
    name_team = df[0].str.slice(stop=-4).str.rstrip()
    df['Team'] = name_team.str.extract(r'(\w+\Z)')
    df['Player'] = name_team.str.split(r'\xa0').str[0]
    df = df[['Player', 'Team', 'Position']]

    # Create ranks
    df['CBS Rank'] = df.index + 1
    pos_rank = df.groupby('Position')['CBS Rank'].rank().astype('int')
    df['CBS Pos'] = df['Position'] + pos_rank.astype('str')
    df = df.drop('Position', axis=1)
    df = df.replace(replacements).replace(sn_teams).replace(espn_teams)
    return df

def ffc_df():
    df = pd.read_csv(r"ffc.csv")
    df = df.rename({'Name': 'Player', 'Pos': 'Position', 'Rank': 'FFC Rank'}, axis=1)
    pos_rank = df.groupby('Position')['FFC Rank'].rank().astype('int')
    df['FFC Pos'] = df['Position'] + pos_rank.astype('str')
    df = df.drop('Position', axis=1)
    df = df.replace(replacements).replace(sn_teams).replace(espn_teams)
    return df

def gen_draft(ffc=ffc_df(), cbs=cbs_df(), espn=espn_df(), sn=sn_df()):
    # Merge
    ds=(ffc.merge(cbs, how='outer', on='Player', suffixes=['_FFC', '_CBS'])
        .merge(espn, how='outer', on='Player', suffixes=[None, '_ESPN'])
        .merge(sn, how='outer', on='Player', suffixes=[None, '_SN']))

    ds['Avg Rank'] = ds[['FFC Rank', 'CBS Rank', 'ESPN Rank', 'SN Rank']].mean(axis=1)
    ds = ds[['Avg Rank', 'FFC Rank', 'CBS Rank', 'ESPN Rank', 'SN Rank', 'Player', 'FFC Pos', 'CBS Pos', 'ESPN Pos', 'SN Pos',
            'Team_FFC', 'Team_CBS', 'Team', 'Team_SN', 'Bye']].sort_values('Avg Rank')

    def check_team(row):
        ffc = row['Team_FFC']
        cbs = row['Team_CBS']
        espn = row['Team']
        sn = row['Team_SN']
        if isinstance(ffc, str):
            return ffc
        elif isinstance(cbs, str):
            return cbs
        elif isinstance(espn, str):
            return espn
        else:
            return sn

    def get_pos(row):
        ffc = row['FFC Pos']
        cbs = row['CBS Pos']
        espn = row['ESPN Pos']
        sn = row['SN Pos']
        if isinstance(ffc, str):
            return re.findall(r'(\D+)\d+', ffc)[0]
        elif isinstance(cbs, str):
            return re.findall(r'(\D+)\d+', cbs)[0]
        elif isinstance(espn, str):
            return re.findall(r'(\D+)\d+', espn)[0]
        else:
            return re.findall(r'(\D+)\d+', sn)[0]

    ds['Team'] = ds.apply(lambda row: check_team(row), axis=1)
    ds = ds.drop(['Team_FFC', 'Team_CBS', 'Team_SN'], axis=1)
    pos_cols = ['FFC Pos', 'CBS Pos', 'ESPN Pos', 'SN Pos']
    ds[pos_cols] = ds[pos_cols].replace({'DEF': 'DST', 'PK': 'K'}, regex=True)
    ds['Pos'] = ds.apply(lambda row: get_pos(row), axis=1)
    ds = ds[['Avg Rank', 'FFC Rank', 'CBS Rank', 'ESPN Rank', 'SN Rank', 'Player', 'Pos', 'FFC Pos', 'CBS Pos', 'ESPN Pos', 'SN Pos',
            'Team', 'Bye']].sort_values('Avg Rank')
    ds['FTeam'] = np.nan
    ds['FPick'] = np.nan

    return ds

ds = gen_draft()

In [41]:
fteams = ['1.0', '2.0', 'RK', 'BD', 'KA', 'BL', 'JB', 'AY', 'BM', 'ES', 'RW', 'NG']
pick_=0
def selection(pick, player, team, df=ds):
    if team not in fteams:
        raise ValueError('Team entered not in list')
    if player.lower() not in list(df['Player'].str.lower()):
        print('WARNING: Player entered not in draft list')
        print('Possible Options: {}'.format(difflib.get_close_matches(player, list(df['Player']))))
        return df
    row = df[df['Player'].str.match(player, case=False)]
    if len(row)>1:
        print('WARNING: Multiple name matches')
    idx = df.index[ds['Player'].str.fullmatch(player, case=False)][0]  # index location
    df.loc[idx,'FTeam'] = team
    df.loc[idx,'FPick'] = pick
    return df

def find_player(player, df=ds):
    player_list = list(df['Player'])
    close_matches = difflib.get_close_matches(player, player_list, n=5)
    contains = [name for name in player_list if player in name]
    print('Close Matches:\n{}'.format(close_matches))
    print('Contains:\n{}'.format(contains))

## Analysis Section
Use this section exclusively for analyzing data

In [299]:
# ds[ds['FTeam']=='JB']
# ds[(ds['FTeam'].isna()) & (ds['Pos']=='WR')].iloc[:40,:]
# ds[ds['FTeam']=='KA']
ds[(ds['FTeam'].isna())].iloc[:40,:]

,Avg Rank,FFC Rank,CBS Rank,ESPN Rank,SN Rank,Player,Pos,FFC Pos,CBS Pos,ESPN Pos,SN Pos,Team,Bye,FTeam,FPick
33,43.250000,34.0,50.0,35.0,54.0,JK Dobbins,RB,RB15,RB20,RB16,RB19,BAL,8.0,NaN,NaN
163,116.000000,NaN,116.0,NaN,NaN,Terrace Marshall Jr.,WR,NaN,WR51,NaN,NaN,CAR,NaN,NaN,NaN
116,117.000000,117.0,NaN,NaN,NaN,Pittsburgh Defense,DST,DST1,NaN,NaN,NaN,PIT,7.0,NaN,NaN
124,120.750000,125.0,93.0,123.0,142.0,Laviska Shenault Jr.,WR,WR41,WR38,WR43,WR51,JAX,7.0,NaN,NaN
164,123.500000,NaN,118.0,NaN,129.0,Jakobi Meyers,WR,NaN,WR53,NaN,WR42,NE,NaN,NaN,NaN
119,126.250000,120.0,143.0,103.0,139.0,David Johnson,RB,RB41,RB44,RB35,RB44,HOU,10.0,NaN,NaN
194,131.500000,NaN,179.0,NaN,84.0,Tyrod Taylor,QB,NaN,QB35,NaN,QB31,HOU,NaN,NaN,NaN
159,131.666667,NaN,111.0,151.0,133.0,T.Y. Hilton,WR,NaN,WR47,WR53,WR46,IND,NaN,NaN,NaN
162,132.666667,NaN,115.0,127.0,156.0,Michael Gallup,WR,NaN,WR50,WR45,WR56,DAL,NaN,NaN,NaN
169,137.666667,NaN,129.0,146.0,138.0,James White,RB,NaN,RB41,RB48,RB43,NE,NaN,NaN,NaN


## Selection Section
Use the following cell to generate and check selections

In [294]:
# Find Player
find_player('Waddle')

Close Matches:
['Jaylen Waddle']
Contains:
['Jaylen Waddle']


In [301]:
fteams = ['1.0', '2.0', 'RK', 'BD', 'KA', 'BL', 'JB', 'AY', 'BM', 'ES', 'RW', 'NG']
pick_+=1
ds = selection(pick_, "Tyler Bass", 'RK')
ds

,Avg Rank,FFC Rank,CBS Rank,ESPN Rank,SN Rank,Player,Pos,FFC Pos,CBS Pos,ESPN Pos,SN Pos,Team,Bye,FTeam,FPick
0,1.500000,1.0,2.0,1.0,2.0,Christian McCaffrey,RB,RB1,RB1,RB1,RB1,CAR,13.0,BL,3.0
1,1.500000,2.0,1.0,2.0,1.0,Patrick Mahomes,QB,QB1,QB1,QB1,QB1,KC,12.0,RK,4.0
2,4.750000,3.0,4.0,3.0,9.0,Dalvin Cook,RB,RB2,RB2,RB2,RB4,MIN,7.0,2.0,26.0
3,4.750000,4.0,6.0,4.0,5.0,Alvin Kamara,RB,RB3,RB3,RB3,RB2,NO,6.0,KA,10.0
4,5.000000,5.0,5.0,6.0,4.0,Josh Allen,QB,QB2,QB3,QB2,QB3,BUF,7.0,JB,6.0
5,7.000000,6.0,7.0,5.0,10.0,Derrick Henry,RB,RB4,RB4,RB4,RB5,TEN,13.0,RW,62.0
9,8.000000,10.0,3.0,11.0,8.0,Lamar Jackson,QB,QB4,QB2,QB4,QB5,BAL,8.0,BM,55.0
7,8.500000,8.0,8.0,7.0,11.0,Ezekiel Elliott,RB,RB5,RB5,RB5,RB6,DAL,7.0,2.0,2.0
6,8.500000,7.0,16.0,8.0,3.0,Kyler Murray,QB,QB3,QB6,QB3,QB2,ARI,12.0,ES,7.0
8,11.500000,9.0,10.0,15.0,12.0,Davante Adams,WR,WR1,WR1,WR1,WR1,GB,13.0,RW,11.0
